In [7]:
import os
import torch
import torchvision
# import torchvision.transforms as transforms
from torchvision import transforms, utils, datasets

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
# from rich import print
# from rich.progress import track
from skimage import io, transform
from collections import Counter

seed = 100
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

batch_size = 32
img_size = 180

if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
    print('Running on GPU: {}.'.format(torch.cuda.get_device_name()))
else:
    print('Running on CPU.')

Running on GPU: Tesla T4.


In [8]:
def load_data_first_time(img_size, desired=None):
    preset_mean = [0.0, 0.0, 0.0]
    preset_std = [0.0, 0.0, 0.0]

    transform = transforms.Compose(
        [
         transforms.Resize(img_size),
         transforms.CenterCrop(img_size),
         transforms.ToTensor()
         ])

    imageDataset = torchvision.datasets.ImageFolder(root='./testdata', transform=transform)
    
    random.shuffle(imageDataset.imgs)


    print ("Sample Images:", imageDataset.imgs[:5])
    print ("Labels:", imageDataset.class_to_idx)

    num_classes = len(imageDataset.classes)

    print ("Number of images:",len(imageDataset))
    
    return imageDataset

imageDataset = load_data_first_time(img_size)
classes = imageDataset.classes

Sample Images: [('./testdata/Data/face41_bbcdb79c-31e9-11eb-bea8-002432182720.jpg', 0), ('./testdata/Data/face325_6655d44b-31ed-11eb-99dd-002432182720.jpg', 0), ('./testdata/Data/face571_034f932b-31f0-11eb-b19e-002432182720.jpg', 0), ('./testdata/Data/face474_f3ced606-31ee-11eb-8481-002432182720.jpg', 0), ('./testdata/Data/face249_619eb34f-31ec-11eb-be8e-002432182720.jpg', 0)]
Labels: {'Data': 0}
Number of images: 5485


In [9]:
def load_fulldata(img_size, desired=None):
    
    means = [0.3831, 0.2346, 0.2189]
    stds =  [0.2026, 0.1548, 0.1604]


    transform = transforms.Compose(
        [
            transforms.Resize(img_size),
            transforms.CenterCrop(img_size),
            transforms.ToTensor(),
            transforms.Normalize(means, stds)
        ])

    fulldataset = datasets.ImageFolder(root='./testdata', transform = transform)
    random.shuffle(fulldataset.imgs)
    
    if desired:
        for i in range(len(fulldataset)):
            img , label = fulldataset.imgs[i]
            if label == desired:
                fulldataset.imgs[i] = (img, 1)
            else:
                fulldataset.imgs[i] = (img, 0)
    
    print (fulldataset.imgs[:15])
    
    

    return fulldataset

def get_loaders(data, batch_size):
    n = len(data)
    
    random.shuffle(data.imgs)

    test_set  = torch.utils.data.Subset(data, range(n))
    
    test_loader  = torch.utils.data.DataLoader(test_set,  batch_size=batch_size, shuffle=True)
    
    return test_loader


In [10]:

def evaluate(model, v, show, loss_fnc, desired=False, sensitivity=False):
    total = 0
    correct = 0
    vloss = 0
    
    ones = torch.ones([batch_size], dtype=torch.long)
    neg_ones = -1 * ones
    twos = ones + 1
    
    TPs = 0.
    FPs = 0.
    FNs = 0.
    
    TP_dict = Counter()
    FN_dict = Counter()
    
    
    
    with torch.no_grad():
        for data in v:
            images, labels = data
            outputs = model(images.type(torch.cuda.FloatTensor))
            labels = labels.type(torch.cuda.LongTensor)

            vloss += loss_fnc(input=outputs.squeeze(), target=labels)


            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)

            correct += (predicted == labels).sum().item()
            
#             print (predicted)
            
            if sensitivity:
                if desired:
                    for i in range(len(predicted)):
                        if predicted[i] == 1:
                            if predicted[i] == labels[i]:
                                TPs += 1
                            else:
                                FNs += 1
                else:
                    for i in range(len(predicted)):
                        if predicted[i] == labels[i]:
                            TP_dict[classes[labels[i]]] += 1
                        else:
                            FN_dict[classes[labels[i]]] += 1
                    
            lookup = classes

            if desired:
                lookup = ["Extra", classes[desired]]

            
            if show:
                for i in range(len(predicted)):
                    if predicted[i] != labels[i]:
                        imshow(images[i])
                        

                        print (">>>>>>", lookup[predicted[i]])
                        print ("Truth:", lookup[labels[i]])
    
    if sensitivity:
        if desired:
            print ("TPs:", TPs, "FNs:", FNs)
            if TPs ==0:
                return 0, vloss.item()/len(v)
            return TPs/(TPs+FNs), vloss.item()/len(v)
#             print (float(correct)/float(total))
            
        else:
            sens = {}
            avg_sens = 0.
            for key in TP_dict.keys():
                sens[key] = float(TP_dict[key])/float(TP_dict[key] + FN_dict[key])
                avg_sens += sens[key]
            print (sens)
            print (float(correct)/float(total))
            
            return avg_sens/float(len(classes)), vloss.item()/len(v)
            
            
            
        
    return float(correct)/float(total), vloss.item()/len(v)

In [11]:
fulldataset = load_fulldata(img_size)
# print (fulldataset)
test_loader = get_loaders(fulldataset, batch_size)

[('./testdata/Data/face284_cc008273-31ec-11eb-b675-002432182720.jpg', 0), ('./testdata/Data/face266_9fc8bfec-31ec-11eb-abbb-002432182720.jpg', 0), ('./testdata/Data/face325_448f702a-31ed-11eb-86e4-002432182720.jpg', 0), ('./testdata/Data/face466_e578f8f2-31ee-11eb-a19a-002432182720.jpg', 0), ('./testdata/Data/face152_1d986fc5-31eb-11eb-9ec3-002432182720.jpg', 0), ('./testdata/Data/face118_b78dfb83-31ea-11eb-a5a3-002432182720.jpg', 0), ('./testdata/Data/face559_c6ad36af-31ef-11eb-b1d3-002432182720.jpg', 0), ('./testdata/Data/face316_3658b6ac-31ed-11eb-9b7a-002432182720.jpg', 0), ('./testdata/Data/face325_6594cac1-31ed-11eb-941f-002432182720.jpg', 0), ('./testdata/Data/face22_77f5e7d3-31e9-11eb-8209-002432182720.jpg', 0), ('./testdata/Data/face301_050627a6-31ed-11eb-a77b-002432182720.jpg', 0), ('./testdata/Data/face573_f20c04f8-31ef-11eb-99fb-002432182720.jpg', 0), ('./testdata/Data/face454_c610c014-31ee-11eb-9f6d-002432182720.jpg', 0), ('./testdata/Data/face561_e7fdd6c6-31ef-11eb-a431-0

In [12]:
class BinaryNet(nn.Module):
    def __init__(self, hidden=32, conv_num=4):
        super(BinaryNet, self).__init__()
        
        self.pool = nn.MaxPool2d(2, 2)
        self.conv_norm1 = nn.BatchNorm2d(20)
        self.conv_norm2 = nn.BatchNorm2d(20)
        self.conv_norm3 = nn.BatchNorm2d(20)
        self.conv_norm4 = nn.BatchNorm2d(20)
        
        self.conv_norm1.track_running_stats=False
        self.conv_norm2.track_running_stats=False
        self.conv_norm3.track_running_stats=False
        self.conv_norm4.track_running_stats=False

        self.conv1 = nn.Conv2d(3,  20, 6)
        self.conv2 = nn.Conv2d(20, 20, 4)
        self.conv3 = nn.Conv2d(20, 20, 3)
        self.conv4 = nn.Conv2d(20, 20, 3)

        self.fc1 = nn.Linear(20*9*9, 400)
        self.lin_norm1 = nn.BatchNorm1d(400)
        self.fc2 = nn.Linear(400, 20)
        self.lin_norm2 = nn.BatchNorm1d(20)
        self.fc3 = nn.Linear(20, 2)
        
        
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv_norm1(self.conv1(x))))
        x = self.pool(F.relu(self.conv_norm2(self.conv2(x))))
        x = self.pool(F.relu(self.conv_norm3(self.conv3(x))))
        x = self.pool(F.relu(self.conv_norm4(self.conv4(x))))

        x = x.view(-1,20*9*9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    

In [13]:
models = [
    torch.load('1.pt2'),
    torch.load('3.pt2'),
    torch.load('4.pt2'),
    torch.load('5.pt2'),
    torch.load('6.pt2')
]
lookup = {
    'Leonard': 1,
    'Penny': 3,
    'Raj': 4,
    'Sheldon': 5,
    'Wolowitz': 6
}
# list_lookup = [1, 3, 4, 5, 6]

list_lookup = [
    'Leonard',
    'Penny',
    'Raj',
    'Sheldon',
    'Wolowitz'
]

In [16]:
onscreen= Counter()
threshold = 0.0
with torch.no_grad():
        for data in test_loader:
            images, _ = data
            bsize = images.shape[0]
            
            outputs = []
            
            for i, model in enumerate(models):
                o = model(images.type(torch.cuda.FloatTensor))
                outputs.append(o)
            
            l = []
            for i, output in enumerate(outputs):
                values, indices = torch.max(output.data, 1)
                
                values = values.to(torch.device("cpu")).numpy()
                indices = indices.to(torch.device("cpu")).numpy()

                l.append(values*indices)
            
            maxes = np.array(l).T
            m = torch.from_numpy(maxes)
            vals, inds = torch.max(m, 1)
#             print (vals)
            # so now we can either just add or we can threshold
            
            for i, ind in enumerate(inds):
                if vals[i] > threshold:
                    onscreen[list_lookup[ind]] +=1
            
                
print (onscreen)

Counter({'Leonard': 1022, 'Wolowitz': 697, 'Raj': 651, 'Sheldon': 633, 'Penny': 208})


In [19]:
import datetime
# from rich import print
for key in onscreen.keys():
    a = datetime.timedelta(seconds=onscreen[key]//2)
    print (key, a)

In [ ]:
Sheldon 0:05:12
Raj 0:05:08
Leonard 0:08:19
Wolowitz 0:05:36
Penny 0:01:35

In [21]:
(1022+697+651+633+208)//2//60

26